In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sales-prediction/products.csv
/kaggle/input/sales-prediction/bills (1).csv
/kaggle/input/sales-prediction/products.csv
/kaggle/input/sales-prediction/bills (1).csv


In [1]:
!pip install openpyxl xlsxwriter


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 3.2 MB/s eta 0:00:00a 0:00:01


In [1]:
import pandas as pd
import ast

def find_column(df, possible_names):
    """
    Finds the correct column name from a list of possibilities,
    handling case and spacing.
    """
    for name in possible_names:
        if name in df.columns.str.strip().tolist():
            return name
    return None

# -----------------------------
# 1️⃣ Load data
# -----------------------------
bills_path = "/kaggle/input/sales-prediction/bills (1).csv"
products_path = "/kaggle/input/sales-prediction/products.csv"

try:
    bills_df = pd.read_csv(bills_path)
    products_df = pd.read_csv(products_path)
except FileNotFoundError as e:
    print(f"Error: {e.filename} not found. Please ensure the CSV files are in the same directory.")
    exit()

# Clean column names by removing leading/trailing spaces
bills_df.columns = bills_df.columns.str.strip()
products_df.columns = products_df.columns.str.strip()

# -----------------------------
# 2️⃣ Prepare Data for Analysis
# -----------------------------
# Find the correct column names using a flexible approach
product_id_col = find_column(bills_df, ['Product_ID', 'Product_IDs', 'Product ID', 'Product IDs'])
quantity_col = find_column(bills_df, ['Quantity', 'Quantities'])
discounted_prices_col = find_column(bills_df, ['Discounted_Prices', 'Discounted Prices'])
product_id_products_col = find_column(products_df, ['ID', 'Product_ID', 'ProductID', 'Product ID'])

if not all([product_id_col, quantity_col, discounted_prices_col, product_id_products_col]):
    print("Error: Could not find one or more required columns.")
    print(f"Bills columns found: {bills_df.columns.tolist()}")
    print(f"Products columns found: {products_df.columns.tolist()}")
    exit()

# Convert the 'Date_Time' column to datetime objects
bills_df['Date_Time'] = pd.to_datetime(bills_df['Date_Time'])

# Convert string-lists to actual lists
bills_df[product_id_col] = bills_df[product_id_col].apply(ast.literal_eval)
bills_df[quantity_col] = bills_df[quantity_col].apply(ast.literal_eval)
bills_df[discounted_prices_col] = bills_df[discounted_prices_col].apply(ast.literal_eval)

# Explode the DataFrame to have one row per product sold
exploded_df = bills_df.explode([product_id_col, quantity_col, discounted_prices_col])

# Rename ProductID column for merging
products_df = products_df.rename(columns={product_id_products_col: 'ProductID'})
exploded_df = exploded_df.rename(columns={product_id_col: 'ProductID', quantity_col: 'Quantity'})

# Merge with the products DataFrame
merged_df = pd.merge(exploded_df, products_df, on='ProductID', how='left')

# Convert relevant columns to numeric type
merged_df['Quantity'] = pd.to_numeric(merged_df['Quantity'])
merged_df['Discounted_Prices'] = pd.to_numeric(merged_df['Discounted_Prices'])

# Calculate total revenue per product in each bill
merged_df['Total_Revenue'] = merged_df['Quantity'] * merged_df['Discounted_Prices']

# -----------------------------
# 3️⃣ Create Summaries and save to CSV
# -----------------------------
# Create time-based columns
merged_df['Date'] = merged_df['Date_Time'].dt.date
merged_df['Week_Start'] = merged_df['Date_Time'].dt.to_period('W').dt.start_time.dt.date
merged_df['Month'] = merged_df['Date_Time'].dt.to_period('M')

# Daily summary
daily_sales_summary = merged_df.groupby(['Date', 'ProductID'], as_index=False)['Quantity'].sum()
daily_sales_summary.rename(columns={'Quantity': 'Total_Quantity_Sold'}, inplace=True)
daily_sales_summary.to_csv("daily_sales_summary.csv", index=False)

# Weekly summary
weekly_sales_summary = merged_df.groupby(['Week_Start', 'ProductID'], as_index=False)['Quantity'].sum()
weekly_sales_summary.rename(columns={'Quantity': 'Total_Quantity_Sold'}, inplace=True)
weekly_sales_summary.to_csv("weekly_sales_summary.csv", index=False)

# Monthly summary
monthly_sales_summary = merged_df.groupby(['Month', 'ProductID'], as_index=False)['Quantity'].sum()
monthly_sales_summary.rename(columns={'Quantity': 'Total_Quantity_Sold'}, inplace=True)
monthly_sales_summary.to_csv("monthly_sales_summary.csv", index=False)

# -----------------------------
# 4️⃣ Brand-level summaries
# -----------------------------
brand_daily = merged_df.groupby(["Date", "Brand Name"], as_index=False)["Total_Revenue"].sum()
brand_daily.to_csv("brand_daily_sales.csv", index=False)

brand_weekly = merged_df.groupby(["Week_Start", "Brand Name"], as_index=False)["Total_Revenue"].sum()
brand_weekly.to_csv("brand_weekly_sales.csv", index=False)

brand_monthly = merged_df.groupby(["Month", "Brand Name"], as_index=False)["Total_Revenue"].sum()
brand_monthly.to_csv("brand_monthly_sales.csv", index=False)

# -----------------------------
# 5️⃣ Identify no-sales products
# -----------------------------
all_sold_products = set(merged_df["ProductID"].unique())
no_sales_products = products_df[~products_df["ProductID"].isin(all_sold_products)]
no_sales_products.to_csv("no_sales_products.csv", index=False)

# -----------------------------
# Final Output
# -----------------------------
print("Sales analysis has been successfully completed and the summaries have been saved to CSV files.")

Sales analysis has been successfully completed and the summaries have been saved to CSV files.


In [1]:
import pandas as pd
import ast
bills_path = "/kaggle/input/sales-prediction/bills (1).csv"
products_path = "/kaggle/input/sales-prediction/products.csv"

try:
    bills_df = pd.read_csv(bills_path)
    products_df = pd.read_csv(products_path)
except FileNotFoundError as e:
    print(f"Error: {e.filename} not found. Please ensure the CSV files are in the same directory.")
    exit()

def find_column(df, possible_names):
    """
    Finds the correct column name from a list of possibilities,
    handling case and spacing.
    """
    for name in possible_names:
        if name in df.columns.str.strip().tolist():
            return name
    return None

def generate_correct_monthly_summary(bills_df, products_df):
    """
    Generates a correct monthly sales summary from the raw bills and products data.
    """
    # Find the correct column names
    product_id_col = find_column(bills_df, ['Product_ID', 'Product_IDs', 'Product ID', 'Product IDs'])
    quantity_col = find_column(bills_df, ['Quantity', 'Quantities'])
    
    if not all([product_id_col, quantity_col]):
        print("Error: Could not find required columns in bills_df for generating a correct summary.")
        return None

    bills_df['Date_Time'] = pd.to_datetime(bills_df['Date_Time'])
    bills_df[product_id_col] = bills_df[product_id_col].apply(ast.literal_eval)
    bills_df[quantity_col] = bills_df[quantity_col].apply(ast.literal_eval)

    exploded_df = bills_df.explode([product_id_col, quantity_col])
    exploded_df = exploded_df.rename(columns={product_id_col: 'ProductID', quantity_col: 'Total_Quantity_Sold'})
    
    products_df.columns = products_df.columns.str.strip()
    products_df = products_df.rename(columns={'ID': 'ProductID'})

    merged_df = pd.merge(exploded_df, products_df, on='ProductID', how='left')
    merged_df['Total_Quantity_Sold'] = pd.to_numeric(merged_df['Total_Quantity_Sold'])
    
    merged_df['Month'] = merged_df['Date_Time'].dt.to_period('M')
    
    correct_summary = merged_df.groupby(['Month', 'ProductID'], as_index=False)['Total_Quantity_Sold'].sum()
    correct_summary['Month'] = correct_summary['Month'].astype(str)
    
    return correct_summary

# -----------------------------
# 1️⃣ Load data
# -----------------------------
try:
    bills_df = pd.read_csv('bills (1).csv')
    products_df = pd.read_csv('products.csv')
    user_summary_df = pd.read_csv('monthly_sales_summary (4).csv')
except FileNotFoundError as e:
    print(f"Error: {e.filename} not found. Please ensure all three CSV files are in the same folder as this script.")
    exit()

# Clean column names in user's file for consistency
user_summary_df.columns = user_summary_df.columns.str.strip()

# -----------------------------
# 2️⃣ Generate a correct summary for comparison
# -----------------------------
correct_summary = generate_correct_monthly_summary(bills_df, products_df)

if correct_summary is None:
    exit()
    
# -----------------------------
# 3️⃣ Compare the two summaries
# -----------------------------
# Merge the correct summary with the user's summary
comparison_df = pd.merge(
    user_summary_df,
    correct_summary,
    on=['Month', 'ProductID'],
    suffixes=('_User', '_Correct')
)

# Identify discrepancies
comparison_df['Match'] = comparison_df['Total_Quantity_Sold_User'] == comparison_df['Total_Quantity_Sold_Correct']

# -----------------------------
# 4️⃣ Print the Accuracy Report
# -----------------------------
total_records = len(comparison_df)
accurate_records = comparison_df['Match'].sum()
inaccurate_records = total_records - accurate_records

print("\n### Monthly Sales Summary Accuracy Report ###")
print("---------------------------------------------")
print(f"Total records in both summaries: {total_records}")
print(f"Accurate records (exact matches): {accurate_records}")
print(f"Inaccurate records (discrepancies): {inaccurate_records}")

if inaccurate_records > 0:
    print("\n### Sample of Incorrect Records ###")
    print(comparison_df[~comparison_df['Match']].head(10).to_string(index=False))
else:
    print("\nAll records match perfectly!")

print("\n---------------------------------------------")

Error: bills (1).csv not found. Please ensure all three CSV files are in the same folder as this script.


NameError: name 'user_summary_df' is not defined

In [10]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from joblib import Parallel, delayed
import numpy as np
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -----------------------------
# Custom Dataset
# -----------------------------
class TimeSeriesDataset(Dataset):
    def __init__(self, series, seq_len):
        self.series = series
        self.seq_len = seq_len

    def __len__(self):
        return len(self.series) - self.seq_len

    def __getitem__(self, idx):
        return (
            torch.tensor(self.series[idx:idx+self.seq_len], dtype=torch.float32).unsqueeze(-1),
            torch.tensor(self.series[idx+self.seq_len], dtype=torch.float32)
        )

# -----------------------------
# GRU with Attention Model
# -----------------------------
class GRUAttention(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=32, num_layers=1):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.attn_fc = nn.Linear(hidden_dim, 1)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        gru_out, _ = self.gru(x)  # (batch, seq_len, hidden_dim)
        attn_weights = torch.softmax(self.attn_fc(gru_out), dim=1)  # (batch, seq_len, 1)
        context_vector = torch.sum(attn_weights * gru_out, dim=1)   # (batch, hidden_dim)
        output = self.fc(context_vector)  # (batch, 1)
        return output

# -----------------------------
# Training + Forecasting Function
# -----------------------------
def train_and_forecast(series, future=30, seq_len=30, epochs=50, batch_size=64):
    scaler = MinMaxScaler()
    series = scaler.fit_transform(series.reshape(-1,1)).flatten()

    dataset = TimeSeriesDataset(series, seq_len)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    model = GRUAttention().to(device)
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Early stopping
    best_loss = float('inf')
    patience = 5
    no_improve = 0

    for epoch in range(epochs):
        epoch_loss = 0
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb).squeeze(-1)
            loss = loss_fn(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(dataloader)
        if avg_loss < best_loss:
            best_loss = avg_loss
            no_improve = 0
        else:
            no_improve += 1

        if no_improve >= patience:
            break

    # Forecast loop
    input_seq = torch.tensor(series[-seq_len:], dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(device)
    preds = []
    model.eval()
    with torch.no_grad():
        for _ in range(future):
            out = model(input_seq).item()
            preds.append(out)
            new_input = torch.cat([input_seq[:,1:,:], torch.tensor([[[out]]], dtype=torch.float32).to(device)], dim=1)
            input_seq = new_input

    preds = scaler.inverse_transform(np.array(preds).reshape(-1,1)).flatten()
    return preds

# -----------------------------
# Forecast per Product
# -----------------------------
def process_product_all(product_id, df, future_daily=30, future_week=7, future_month=30, seq_len=30):
    group = df[df["ProductID"] == product_id].sort_values("Date")
    series = group["Total_Quantity_Sold"].values
    if len(series) < seq_len:
        return None

    daily_forecast = train_and_forecast(series, future=future_daily, seq_len=seq_len)
    week_forecast = train_and_forecast(series, future=future_week, seq_len=seq_len)
    month_forecast = train_and_forecast(series, future=future_month, seq_len=seq_len)

    return {
        "ProductID": product_id,
        "Daily_Forecast": daily_forecast.tolist(),
        "Week_Forecast": week_forecast.tolist(),
        "Month_Forecast": month_forecast.tolist()
    }

# -----------------------------
# Run Forecasting in Parallel
# -----------------------------
product_ids = summary_daily_df["ProductID"].unique()
results = Parallel(n_jobs=-1)(
    delayed(process_product_all)(pid, summary_daily_df) for pid in product_ids
)
results = [r for r in results if r is not None]

forecast_df = pd.DataFrame(results)
forecast_df = forecast_df.merge(products_df[['ProductID', 'Brand Name']], on='ProductID', how='left')
forecast_df.to_csv("product_forecasts_all.csv", index=False)
print("✅ Product-level forecasts saved (daily, weekly, monthly)")

# -----------------------------
# BRAND-LEVEL FORECASTS
# -----------------------------
def aggregate_brand_forecast(forecast_df, col_name, horizon_name):
    expanded = []
    for _, row in forecast_df.iterrows():
        for i, val in enumerate(row[col_name]):
            expanded.append({"Brand": row['Brand Name'], horizon_name: i+1, "Forecast_Qty": val})
    expanded_df = pd.DataFrame(expanded)
    brand_forecast = expanded_df.groupby(["Brand", horizon_name], as_index=False)["Forecast_Qty"].sum()
    return brand_forecast

# Daily
brand_daily_forecast = aggregate_brand_forecast(forecast_df, "Daily_Forecast", "Day")
brand_daily_forecast.to_csv("brand_daily_forecast.csv", index=False)

# Weekly
brand_week_forecast = aggregate_brand_forecast(forecast_df, "Week_Forecast", "Week")
brand_week_forecast.to_csv("brand_week_forecast.csv", index=False)

# Monthly
brand_month_forecast = aggregate_brand_forecast(forecast_df, "Month_Forecast", "Month")
brand_month_forecast.to_csv("brand_month_forecast.csv", index=False)

print("✅ Brand-level forecasts saved (daily, weekly, monthly)")


✅ Product-level forecasts saved (daily, weekly, monthly)
✅ Brand-level forecasts saved (daily, weekly, monthly)


In [4]:
import pandas as pd
import ast
from datetime import datetime
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from joblib import Parallel, delayed
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -----------------------------
# 1️⃣ Load Data
# -----------------------------
bills_path = "/kaggle/input/sales-prediction/bills (1).csv"
products_path = "/kaggle/input/sales-prediction/products.csv"

bills_df = pd.read_csv(bills_path)
products_df = pd.read_csv(products_path)
products_df["ProductID"] = products_df["ID"].astype(str)

# -----------------------------
# 2️⃣ Parse and Explode Data
# -----------------------------
bills_df['Product_IDs'] = bills_df['Product_IDs'].apply(ast.literal_eval)
bills_df['Quantities'] = bills_df['Quantities'].apply(ast.literal_eval)
bills_df['Date'] = pd.to_datetime(bills_df['Date_Time']).dt.date

exploded_df = bills_df.explode(['Product_IDs', 'Quantities'])
exploded_df['ProductID'] = exploded_df['Product_IDs'].astype(str)
exploded_df['Quantity_Sold'] = exploded_df['Quantities'].astype(int)

# -----------------------------
# 3️⃣ Aggregate Daily Sales
# -----------------------------
summary_daily_df = (
    exploded_df.groupby(['Date', 'ProductID'], as_index=False)['Quantity_Sold']
    .sum()
    .rename(columns={'Quantity_Sold': 'Total_Quantity_Sold'})
)

# -----------------------------
# 4️⃣ Identify No-Sales Products
# -----------------------------
all_sold_products = set(summary_daily_df["ProductID"].unique())
no_sales_products_df = products_df[~products_df["ProductID"].isin(all_sold_products)]
no_sales_products_df.to_csv("no_sales_products.csv", index=False)

# Save daily sales summary
summary_daily_df.to_csv("summary_daily_sales.csv", index=False)

# -----------------------------
# 5️⃣ Brand-Level Summaries
# -----------------------------
summary_daily_df = summary_daily_df.merge(products_df[['ProductID', 'Brand Name']], on='ProductID', how='left')

brand_daily = summary_daily_df.groupby(["Date", "Brand Name"], as_index=False)["Total_Quantity_Sold"].sum()
brand_daily.to_csv("brand_daily_sales.csv", index=False)

# -----------------------------
# 6️⃣ Forecasting Model Definition
# -----------------------------
class TimeSeriesDataset(Dataset):
    def __init__(self, series, seq_len):
        self.series = series
        self.seq_len = seq_len

    def __len__(self):
        return len(self.series) - self.seq_len

    def __getitem__(self, idx):
        return (
            torch.tensor(self.series[idx:idx+self.seq_len], dtype=torch.float32).unsqueeze(-1),
            torch.tensor(self.series[idx+self.seq_len], dtype=torch.float32)
        )

class GRUAttention(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=32, num_layers=1):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.attn_fc = nn.Linear(hidden_dim, 1)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        gru_out, _ = self.gru(x)
        attn_weights = torch.softmax(self.attn_fc(gru_out), dim=1)
        context_vector = torch.sum(attn_weights * gru_out, dim=1)
        output = self.fc(context_vector)
        return output

def train_and_forecast(series, future=30, seq_len=30, epochs=50, batch_size=64):
    scaler = MinMaxScaler()
    series = scaler.fit_transform(series.reshape(-1,1)).flatten()

    dataset = TimeSeriesDataset(series, seq_len)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    model = GRUAttention().to(device)
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    best_loss = float('inf')
    patience = 5
    no_improve = 0

    for epoch in range(epochs):
        epoch_loss = 0
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb).squeeze(-1)
            loss = loss_fn(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(dataloader)
        if avg_loss < best_loss:
            best_loss = avg_loss
            no_improve = 0
        else:
            no_improve += 1
        if no_improve >= patience:
            break

    input_seq = torch.tensor(series[-seq_len:], dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(device)
    preds = []
    model.eval()
    with torch.no_grad():
        for _ in range(future):
            out = model(input_seq).item()
            preds.append(out)
            new_input = torch.cat([input_seq[:,1:,:], torch.tensor([[[out]]], dtype=torch.float32).to(device)], dim=1)
            input_seq = new_input

    preds = scaler.inverse_transform(np.array(preds).reshape(-1,1)).flatten()
    return preds

# -----------------------------
# 7️⃣ Forecast Per Product
# -----------------------------
def process_product_all(product_id, df, future_daily=30, future_week=7, future_month=30, seq_len=30):
    group = df[df["ProductID"] == product_id].sort_values("Date")
    series = group["Total_Quantity_Sold"].values
    if len(series) < seq_len:
        return None

    daily_forecast = train_and_forecast(series, future=future_daily, seq_len=seq_len)
    week_forecast = train_and_forecast(series, future=future_week, seq_len=seq_len)
    month_forecast = train_and_forecast(series, future=future_month, seq_len=seq_len)

    return {
        "ProductID": product_id,
        "Daily_Forecast": daily_forecast.tolist(),
        "Week_Forecast": week_forecast.tolist(),
        "Month_Forecast": month_forecast.tolist()
    }

product_ids = summary_daily_df["ProductID"].unique()
results = Parallel(n_jobs=-1)(
    delayed(process_product_all)(pid, summary_daily_df) for pid in product_ids
)
results = [r for r in results if r is not None]

forecast_df = pd.DataFrame(results)
forecast_df = forecast_df.merge(products_df[['ProductID', 'Brand Name']], on='ProductID', how='left')
forecast_df.to_csv("product_forecasts_all.csv", index=False)

# -----------------------------
# 8️⃣ Aggregate Brand Forecasts
# -----------------------------
def aggregate_brand_forecast(forecast_df, col_name, horizon_name):
    expanded = []
    for _, row in forecast_df.iterrows():
        for i, val in enumerate(row[col_name]):
            expanded.append({"Brand": row['Brand Name'], horizon_name: i+1, "Forecast_Qty": val})
    expanded_df = pd.DataFrame(expanded)
    brand_forecast = expanded_df.groupby(["Brand", horizon_name], as_index=False)["Forecast_Qty"].sum()
    return brand_forecast

brand_daily_forecast = aggregate_brand_forecast(forecast_df, "Daily_Forecast", "Day")
brand_daily_forecast.to_csv("brand_daily_forecast.csv", index=False)

brand_week_forecast = aggregate_brand_forecast(forecast_df, "Week_Forecast", "Week")
brand_week_forecast.to_csv("brand_week_forecast.csv", index=False)

brand_month_forecast = aggregate_brand_forecast(forecast_df, "Month_Forecast", "Month")
brand_month_forecast.to_csv("brand_month_forecast.csv", index=False)

print("✅ All steps completed successfully.")

✅ All steps completed successfully.


In [36]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from joblib import Parallel, delayed
import ast

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -----------------------------
# Load Data
# -----------------------------
bills_df = pd.read_csv("/kaggle/input/sales-prediction/bills (1).csv")
products_df = pd.read_csv("/kaggle/input/sales-prediction/products.csv")

# -----------------------------
# Safe Preprocessing & Merge
# -----------------------------
bills_df.columns = bills_df.columns.str.strip()

# Expand multiple Product IDs
bills_df["Product_IDs"] = bills_df["Product_IDs"].astype(str)
bills_df = bills_df.assign(ProductID=bills_df["Product_IDs"].str.split(",")).explode("ProductID")
bills_df["ProductID"] = bills_df["ProductID"].str.strip()

# Convert stringified lists to Python lists for quantities
bills_df["Product_IDs"] = bills_df["Product_IDs"].apply(ast.literal_eval)
bills_df["Quantities"] = bills_df["Quantities"].apply(ast.literal_eval)

# Explode each product and quantity
bills_df = bills_df.explode(["Product_IDs", "Quantities"])
bills_df["ProductID"] = bills_df["Product_IDs"].astype(str).str.strip()
bills_df["Quantity_Sold"] = bills_df["Quantities"].astype(int)

# -----------------------------
# Ensure products_df columns
# -----------------------------
products_df.rename(columns={
    'Product Code': 'ProductID',
    'Product Name': 'Product_Name',
    'Brand Name': 'Brand Name',
    'Price (INR)': 'Price'
}, inplace=True)

products_df["ProductID"] = products_df["ProductID"].astype(str).str.strip()

# Merge bills with products
merged_df = bills_df.merge(products_df[['ProductID', 'Product_Name', 'Brand Name']], on="ProductID", how="left")
merged_df["Date"] = pd.to_datetime(merged_df["Date_Time"]).dt.date

print("✅ Merged shape:", merged_df.shape)

# -----------------------------
# Prepare Daily Summary
# -----------------------------
summary_daily_df = (
    merged_df.groupby(["Date", "ProductID", "Brand Name"])["Quantity_Sold"]
    .sum()
    .reset_index()
    .rename(columns={"Quantity_Sold": "Total_Quantity_Sold"})
)

# -----------------------------
# TimeSeries Dataset
# -----------------------------
class TimeSeriesDataset(Dataset):
    def __init__(self, series, seq_len):
        series = np.array(series, dtype=float)
        if len(series) <= seq_len:
            series = np.pad(series, (0, seq_len + 1 - len(series)), mode="constant", constant_values=0.1)
        self.series = series
        self.seq_len = seq_len

    def __len__(self):
        return max(len(self.series) - self.seq_len, 1)

    def __getitem__(self, idx):
        idx = min(idx, len(self.series) - self.seq_len - 1)
        return (
            torch.tensor(self.series[idx:idx+self.seq_len], dtype=torch.float32).unsqueeze(-1),
            torch.tensor(self.series[idx+self.seq_len], dtype=torch.float32)
        )

# -----------------------------
# GRU + Attention Model
# -----------------------------
class GRUAttention(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=32, num_layers=1):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.attn_fc = nn.Linear(hidden_dim, 1)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        gru_out, _ = self.gru(x)
        attn_weights = torch.softmax(self.attn_fc(gru_out), dim=1)
        context_vector = torch.sum(attn_weights * gru_out, dim=1)
        output = self.fc(context_vector)
        return output

# -----------------------------
# Train and Forecast
# -----------------------------
def train_and_forecast(series, future=30, seq_len=30, epochs=50, batch_size=64):
    if np.sum(series) == 0:
        return np.zeros(future)

    scaler = MinMaxScaler()
    series_scaled = scaler.fit_transform(series.reshape(-1,1)).flatten()

    dataset = TimeSeriesDataset(series_scaled, seq_len)
    batch_size = min(batch_size, len(dataset))
    shuffle = len(dataset) > 1
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

    model = GRUAttention().to(device)
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    best_loss = float('inf')
    patience, no_improve = 5, 0

    for epoch in range(epochs):
        epoch_loss = 0
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb).squeeze(-1)
            loss = loss_fn(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        avg_loss = epoch_loss / len(dataloader)
        if avg_loss < best_loss:
            best_loss = avg_loss
            no_improve = 0
        else:
            no_improve += 1
        if no_improve >= patience:
            break

    # Forecast
    model.eval()
    input_seq = torch.tensor(series_scaled[-seq_len:], dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(device)
    preds = []
    with torch.no_grad():
        for _ in range(future):
            out = model(input_seq).item()
            preds.append(out)
            new_input = torch.cat([input_seq[:,1:,:], torch.tensor([[[out]]], dtype=torch.float32).to(device)], dim=1)
            input_seq = new_input

    preds = scaler.inverse_transform(np.array(preds).reshape(-1,1)).flatten()
    preds = np.maximum(preds, 0)
    return np.round(preds)

# -----------------------------
# Product-level Forecasts
# -----------------------------
def process_product_all(product_id, df, seq_len=30):
    group = df[df["ProductID"] == product_id].sort_values("Date")
    series = group["Total_Quantity_Sold"].values
    if len(series) < 2:
        return None
    return {
        "ProductID": product_id,
        "Brand Name": group["Brand Name"].iloc[0],
        "Daily_Forecast": train_and_forecast(series, future=30, seq_len=seq_len).tolist(),
        "Weekly_Forecast": train_and_forecast(series, future=7, seq_len=seq_len).tolist(),
        "Monthly_Forecast": train_and_forecast(series, future=30, seq_len=seq_len).tolist()
    }

product_ids = summary_daily_df["ProductID"].unique()
results = Parallel(n_jobs=-1)(
    delayed(process_product_all)(pid, summary_daily_df) for pid in product_ids
)
results = [r for r in results if r is not None]

forecast_df = pd.DataFrame(results)
forecast_df.to_csv("product_forecasts_all.csv", index=False)
print("✅ Product-level forecasts saved")

# -----------------------------
# Brand-level Forecasts (aggregate across products)
# -----------------------------
def aggregate_brand_forecast(forecast_df, col_name, horizon_name):
    expanded = []
    for _, row in forecast_df.iterrows():
        brand_col = row.get('Brand Name', 'Unknown')
        for i, val in enumerate(row[col_name]):
            expanded.append({"Brand": brand_col, horizon_name: i+1, "Forecast_Qty": val})
    expanded_df = pd.DataFrame(expanded)
    brand_forecast = expanded_df.groupby(["Brand", horizon_name], as_index=False)["Forecast_Qty"].sum()
    brand_forecast["Forecast_Qty"] = brand_forecast["Forecast_Qty"].round()
    return brand_forecast

brand_daily_forecast = aggregate_brand_forecast(forecast_df, "Daily_Forecast", "Day")
brand_weekly_forecast = aggregate_brand_forecast(forecast_df, "Weekly_Forecast", "Week")
brand_monthly_forecast = aggregate_brand_forecast(forecast_df, "Monthly_Forecast", "Month")

brand_daily_forecast.to_csv("brand_daily_forecast.csv", index=False)
brand_weekly_forecast.to_csv("brand_weekly_forecast.csv", index=False)
brand_monthly_forecast.to_csv("brand_monthly_forecast.csv", index=False)
print("✅ Brand-level forecasts saved (aggregated across products)")


✅ Merged shape: (5398837, 16)
✅ Product-level forecasts saved


KeyError: 'Brand'

In [9]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

# -------------------------
# 1. Load Data
# -------------------------
bills_df = pd.read_csv("/kaggle/input/trail-bills/10000.csv")
products_df = pd.read_csv("/kaggle/input/sales-prediction/products.csv")

# Clean column names
bills_df.columns = bills_df.columns.str.strip()
products_df.columns = products_df.columns.str.strip()

# Force rename the last column in products_df → "ProductID"
products_df.rename(columns={products_df.columns[-1]: "ProductID"}, inplace=True)

# -------------------------
# Process Bills Data
# -------------------------
# Expand multiple product IDs in one bill
bills_df["Product_IDs"] = bills_df["Product_IDs"].astype(str)
bills_df = bills_df.assign(ProductID=bills_df["Product_IDs"].str.split(",")).explode("ProductID")
bills_df["ProductID"] = bills_df["ProductID"].str.strip()

# -------------------------
# Merge with Products
# -------------------------
products_df["ProductID"] = products_df["ProductID"].astype(str).str.strip()
merged_df = bills_df.merge(products_df, on="ProductID", how="left")
merged_df["Date"] = pd.to_datetime(merged_df["Date_Time"]).dt.date

print("✅ Merged shape:", merged_df.shape)
print(merged_df.head())

# -------------------------
# 3. Aggregate Daily Sales
# -------------------------
daily_sales = (
    merged_df.groupby(["Date", "ProductID", "Product Name", "Brand Name"])
    .agg(Total_Sales=("Total_Amount", "sum"))
    .reset_index()
)

# -------------------------
# 4. PyTorch Dataset
# -------------------------
class SalesDataset(Dataset):
    def __init__(self, series, seq_len=7):
        self.scaler = MinMaxScaler()
        self.series = self.scaler.fit_transform(series.reshape(-1, 1))
        self.seq_len = seq_len

    def __len__(self):
        return len(self.series) - self.seq_len

    def __getitem__(self, idx):
        X = self.series[idx:idx+self.seq_len]
        y = self.series[idx+self.seq_len]
        return torch.FloatTensor(X), torch.FloatTensor(y)

# -------------------------
# 5. Simple LSTM Model
# -------------------------
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

# -------------------------
# 6. Train & Forecast
# -------------------------
def train_and_forecast(series, days=30, seq_len=7, epochs=20):
    if len(series) < seq_len + 1:
        return [0] * days  # not enough data

    dataset = SalesDataset(series, seq_len)
    loader = DataLoader(dataset, batch_size=16, shuffle=True)

    model = LSTMModel()
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    # Training
    for epoch in range(epochs):
        for X, y in loader:
            X = X.unsqueeze(-1)  # [batch, seq_len, 1]
            y = y.unsqueeze(-1)
            optimizer.zero_grad()
            output = model(X)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()

    # Forecasting
    scaler = dataset.scaler
    last_seq = dataset.series[-seq_len:]
    preds = []

    for _ in range(days):
        inp = torch.FloatTensor(last_seq).unsqueeze(0).unsqueeze(-1)
        pred = model(inp).item()
        preds.append(pred)
        last_seq = np.append(last_seq[1:], pred)

    preds = scaler.inverse_transform(np.array(preds).reshape(-1, 1)).flatten()
    return preds.tolist()

# -------------------------
# 7. Forecast Products
# -------------------------
product_forecasts = []
for pid, group in daily_sales.groupby("ProductID"):
    series = group.sort_values("Date")["Total_Sales"].values
    preds = train_and_forecast(series)

    weekly_avg = round(np.mean(preds[:7]), 2)
    monthly_avg = round(np.mean(preds[:30]), 2)

    product_forecasts.append({
        "ProductID": pid,
        "Product Name": group["Product Name"].iloc[0],
        "Predictions": preds,
        "Weekly_Avg": weekly_avg,
        "Monthly_Avg": monthly_avg
    })

product_forecasts_df = pd.DataFrame(product_forecasts)
product_forecasts_df.to_csv("product_forecasts.csv", index=False)

# -------------------------
# 8. Forecast Brands
# -------------------------
brand_forecasts = []
for (pid, brand), group in daily_sales.groupby(["ProductID", "Brand Name"]):
    series = group.sort_values("Date")["Total_Sales"].values
    preds = train_and_forecast(series)

    weekly_avg = round(np.mean(preds[:7]), 2)
    monthly_avg = round(np.mean(preds[:30]), 2)

    brand_forecasts.append({
        "ProductID": pid,
        "Brand Name": brand,
        "Predictions": preds,
        "Weekly_Avg": weekly_avg,
        "Monthly_Avg": monthly_avg
    })

brand_forecasts_df = pd.DataFrame(brand_forecasts)
brand_forecasts_df.to_csv("brand_forecasts.csv", index=False)

print("✅ Forecasting complete. Results saved to product_forecasts.csv and brand_forecasts.csv")


✅ Merged shape: (39704, 22)
     Bill_ID  Mobile_Number            Date_Time  \
0  BILL00001     9503299454  2024-07-06 01:09:51   
1  BILL00001     9503299454  2024-07-06 01:09:51   
2  BILL00001     9503299454  2024-07-06 01:09:51   
3  BILL00002     9518832283  2022-11-25 05:51:28   
4  BILL00002     9518832283  2022-11-25 05:51:28   

                                         Product_IDs       Quantities  \
0  ['PVSL0504031125', 'PVSL0502021025', 'PVSL0516...        [9, 2, 4]   
1  ['PVSL0504031125', 'PVSL0502021025', 'PVSL0516...        [9, 2, 4]   
2  ['PVSL0504031125', 'PVSL0502021025', 'PVSL0516...        [9, 2, 4]   
3  ['PVSL0905031125', 'PVSL0306030126', 'PVSL0114...  [5, 5, 3, 5, 3]   
4  ['PVSL0905031125', 'PVSL0306030126', 'PVSL0114...  [5, 5, 3, 5, 3]   

                         Original_Prices                    Offers_Applied  \
0         [36319.38, 19292.36, 35040.03]              ['Yes', 'No', 'Yes']   
1         [36319.38, 19292.36, 35040.03]              ['Yes', 'N

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [42]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from joblib import Parallel, delayed
import ast

# Use a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------------------
# 1. Load Data
# ----------------------------
# Load the bills and products datasets
bills_df = pd.read_csv("/kaggle/input/sales-prediction/bills (1).csv")
products_df = pd.read_csv("/kaggle/input/sales-prediction/products.csv")

# ----------------------------
# 2. Preprocess Bills Data
# ----------------------------
# Clean column names by stripping whitespace
bills_df.columns = bills_df.columns.str.strip()

# Convert Product_IDs and Quantities from string representations to lists
bills_df["Product_IDs"] = bills_df["Product_IDs"].astype(str)
bills_df["Quantities"] = bills_df["Quantities"].apply(ast.literal_eval)

# Explode the DataFrame to have one row per product in each bill
bills_df = bills_df.assign(ProductID=bills_df["Product_IDs"].str.split(",")).explode("ProductID")
bills_df = bills_df.explode(["Quantities"])

# Clean up ProductID and set data types
bills_df["ProductID"] = bills_df["ProductID"].astype(str).str.strip()
bills_df["Quantity_Sold"] = bills_df["Quantities"].astype(int)
bills_df["Date"] = pd.to_datetime(bills_df["Date_Time"]).dt.date

# ----------------------------
# 3. Preprocess Products Data
# ----------------------------
# Rename columns for consistency
products_df.rename(columns={
    'Product Code': 'ProductID',
    'Product Name': 'Product_Name',
    'Brand Name': 'Brand Name',
    'Price (INR)': 'Price'
}, inplace=True)
products_df["ProductID"] = products_df["ProductID"].astype(str).str.strip()

# ----------------------------
# 4. Merge DataFrames
# ----------------------------
# Merge the bills and products dataframes
merged_df = bills_df.merge(products_df[['ProductID', 'Product_Name', 'Brand Name']], on='ProductID', how='left')
# Fill any missing brand names with 'Unknown'
merged_df['Brand Name'] = merged_df['Brand Name'].fillna('Unknown')

# ----------------------------
# 5. Create Daily Sales Summary
# ----------------------------
# Group by Date, ProductID, and Brand Name to get total quantity sold each day
summary_daily_df = (
    merged_df.groupby(["Date", "ProductID", "Brand Name"])["Quantity_Sold"]
    .sum()
    .reset_index()
    .rename(columns={"Quantity_Sold": "Total_Quantity_Sold"})
)

# ----------------------------
# 6. PyTorch Dataset for Time Series
# ----------------------------
class TimeSeriesDataset(Dataset):
    def __init__(self, series, seq_len):
        series = np.array(series, dtype=float)
        # Pad the series if it's shorter than the sequence length
        if len(series) <= seq_len:
            series = np.pad(series, (0, seq_len + 1 - len(series)), 'constant', constant_values=0.1)
        self.series = series
        self.seq_len = seq_len

    def __len__(self):
        return max(len(self.series) - self.seq_len, 1)

    def __getitem__(self, idx):
        # Ensure the index is within bounds
        idx = min(idx, len(self.series) - self.seq_len - 1)
        return (
            torch.tensor(self.series[idx:idx+self.seq_len], dtype=torch.float32).unsqueeze(-1),
            torch.tensor(self.series[idx+self.seq_len], dtype=torch.float32)
        )

# ----------------------------
# 7. GRU with Attention Model
# ----------------------------
class GRUAttention(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=32, num_layers=1):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.attn_fc = nn.Linear(hidden_dim, 1)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        gru_out, _ = self.gru(x)
        attn_weights = torch.softmax(self.attn_fc(gru_out), dim=1)
        context_vector = torch.sum(attn_weights * gru_out, dim=1)
        output = self.fc(context_vector)
        return output

# ----------------------------
# 8. Training and Forecasting Function
# ----------------------------
def train_and_forecast(series, future=30, seq_len=30, epochs=50, batch_size=64):
    # If there's no sales data, predict zeros
    if np.sum(series) == 0 or len(series) == 0:
        return np.zeros(future).tolist()

    # Scale the data
    scaler = MinMaxScaler()
    series_scaled = scaler.fit_transform(series.reshape(-1, 1)).flatten()

    # Create dataset and dataloader
    dataset = TimeSeriesDataset(series_scaled, seq_len)
    batch_size = min(batch_size, len(dataset))
    shuffle = len(dataset) > 1
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

    # Initialize model, loss function, and optimizer
    model = GRUAttention().to(device)
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Training loop with early stopping
    best_loss = float('inf')
    patience, no_improve = 5, 0
    for epoch in range(epochs):
        epoch_loss = 0
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb).squeeze(-1)
            loss = loss_fn(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(dataloader)
        if avg_loss < best_loss:
            best_loss = avg_loss
            no_improve = 0
        else:
            no_improve += 1
        if no_improve >= patience:
            break

    # Forecasting
    model.eval()
    input_seq = torch.tensor(series_scaled[-seq_len:], dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(device)
    preds = []
    with torch.no_grad():
        for _ in range(future):
            out = model(input_seq).item()
            preds.append(out)
            new_input = torch.cat([input_seq[:, 1:, :], torch.tensor([[[out]]], dtype=torch.float32).to(device)], dim=1)
            input_seq = new_input

    # Inverse transform the predictions to get the actual scale
    preds = scaler.inverse_transform(np.array(preds).reshape(-1, 1)).flatten()
    preds = np.maximum(preds, 0)
    return np.round(preds).tolist()

# ----------------------------
# 9. Generate Forecasts per Product
# ----------------------------
def process_product_all(product_id, df, seq_len=30):
    group = df[df["ProductID"] == product_id].sort_values("Date")
    series = group["Total_Quantity_Sold"].values
    # Pad the series if it's too short
    if len(series) < 2:
        series = np.pad(series, (0, 2 - len(series)), 'constant', constant_values=0.1)

    # Generate daily, weekly, and monthly forecasts
    return {
        "ProductID": product_id,
        "Brand Name": group["Brand Name"].iloc[0],
        "Daily_Forecast": train_and_forecast(series, future=30, seq_len=seq_len),
        "Weekly_Forecast": train_and_forecast(series, future=7, seq_len=seq_len),
        "Monthly_Forecast": train_and_forecast(series, future=30, seq_len=seq_len)
    }

# Run the forecasting in parallel for speed
product_ids = summary_daily_df["ProductID"].unique()
results = Parallel(n_jobs=-1)(delayed(process_product_all)(pid, summary_daily_df) for pid in product_ids)
results = [r for r in results if r is not None]

# Save the product-level forecasts
forecast_df = pd.DataFrame(results)
forecast_df.to_csv("product_forecasts_all.csv", index=False)
print("✅ Product-level forecasts saved to product_forecasts_all.csv")

# ----------------------------
# 10. Aggregate Forecasts by Brand
# ----------------------------
def aggregate_brand_forecast(forecast_df, col_name, horizon_name):
    expanded = []
    for _, row in forecast_df.iterrows():
        # Handle cases where brand name might be null
        brand_col = row['Brand Name'] if pd.notnull(row['Brand Name']) else 'Unknown'
        for i, val in enumerate(row[col_name]):
            expanded.append({"Brand": brand_col, horizon_name: i + 1, "Forecast_Qty": val})

    expanded_df = pd.DataFrame(expanded)
    # Group by brand and the time horizon (Day, Week, Month)
    brand_forecast = expanded_df.groupby(["Brand", horizon_name], as_index=False)["Forecast_Qty"].sum()
    brand_forecast["Forecast_Qty"] = brand_forecast["Forecast_Qty"].round()
    return brand_forecast

# Generate and save brand-level forecasts
brand_daily_forecast = aggregate_brand_forecast(forecast_df, "Daily_Forecast", "Day")
brand_weekly_forecast = aggregate_brand_forecast(forecast_df, "Weekly_Forecast", "Week")
brand_monthly_forecast = aggregate_brand_forecast(forecast_df, "Monthly_Forecast", "Month")

brand_daily_forecast.to_csv("brand_daily_forecast.csv", index=False)
brand_weekly_forecast.to_csv("brand_weekly_forecast.csv", index=False)
brand_monthly_forecast.to_csv("brand_monthly_forecast.csv", index=False)
print("✅ Brand-level forecasts saved to brand_daily_forecast.csv, brand_weekly_forecast.csv, and brand_monthly_forecast.csv")

✅ Product-level forecasts saved to product_forecasts_all.csv
✅ Brand-level forecasts saved to brand_daily_forecast.csv, brand_weekly_forecast.csv, and brand_monthly_forecast.csv


In [44]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from joblib import Parallel, delayed
import ast

# Use a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------------------
# 1. Load Data
# ----------------------------
# Load the bills and products datasets
bills_df = pd.read_csv("/kaggle/input/sales-prediction/bills (1).csv")
products_df = pd.read_csv("/kaggle/input/sales-prediction/products.csv")

# ----------------------------
# 2. Preprocess Bills Data
# ----------------------------
# Clean column names by stripping whitespace
bills_df.columns = bills_df.columns.str.strip()

# Convert Product_IDs and Quantities from string representations to lists
bills_df["Product_IDs"] = bills_df["Product_IDs"].apply(ast.literal_eval)
bills_df["Quantities"] = bills_df["Quantities"].apply(ast.literal_eval)


# Explode the DataFrame to have one row per product-quantity pair
bills_df = bills_df.explode(["Product_IDs", "Quantities"])
bills_df.rename(columns={"Product_IDs": "ProductID", "Quantities": "Quantity_Sold"}, inplace=True)


bills_df["ProductID"] = bills_df["ProductID"].astype(str).str.strip()
bills_df["Quantity_Sold"] = bills_df["Quantity_Sold"].astype(int)

bills_df["Date"] = pd.to_datetime(bills_df["Date_Time"]).dt.date

# ----------------------------
# 3. Preprocess Products Data
# ----------------------------
# Clean up column names and ProductID
products_df.columns = products_df.columns.str.strip()
products_df.rename(columns={'Product Code': 'ProductID'}, inplace=True)
products_df["ProductID"] = products_df["ProductID"].astype(str).str.strip()


# ----------------------------
# 4. Merge DataFrames
# ----------------------------
# Merge the bills and products dataframes
merged_df = bills_df.merge(products_df[['ProductID', 'Brand Name']], on='ProductID', how='left')
merged_df['Brand Name'] = merged_df['Brand Name'].fillna('Unknown')

# ----------------------------
# 5. Create Daily Sales Summary
# ----------------------------
summary_daily_df = (
    merged_df.groupby(["Date", "ProductID", "Brand Name"])["Quantity_Sold"]
    .sum()
    .reset_index()
    .rename(columns={"Quantity_Sold": "Total_Quantity_Sold"})
)

# ----------------------------
# 6. PyTorch Dataset for Time Series
# ----------------------------
class TimeSeriesDataset(Dataset):
    def __init__(self, series, seq_len):
        self.series = series
        self.seq_len = seq_len

    def __len__(self):
        return len(self.series) - self.seq_len

    def __getitem__(self, idx):
        return (
            torch.tensor(self.series[idx:idx+self.seq_len], dtype=torch.float32).unsqueeze(-1),
            torch.tensor(self.series[idx+self.seq_len], dtype=torch.float32)
        )

# ----------------------------
# 7. GRU with Attention Model
# ----------------------------
class GRUAttention(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=32, num_layers=1):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.attn_fc = nn.Linear(hidden_dim, 1)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        gru_out, _ = self.gru(x)
        attn_weights = torch.softmax(self.attn_fc(gru_out), dim=1)
        context_vector = torch.sum(attn_weights * gru_out, dim=1)
        output = self.fc(context_vector)
        return output

# ----------------------------
# 8. Training, Forecasting, and Accuracy Check
# ----------------------------
def train_and_forecast_with_accuracy(series, future=30, seq_len=30, epochs=50, batch_size=64):
    if len(series) < seq_len + 10: # Ensure enough data for a meaningful split
        return np.zeros(future).tolist(), 0.0

    # Train-test split
    train_series, test_series = train_test_split(series, test_size=0.2, shuffle=False)

    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_series.reshape(-1, 1)).flatten()

    train_dataset = TimeSeriesDataset(train_scaled, seq_len)
    if len(train_dataset) == 0:
        return np.zeros(future).tolist(), 0.0
    
    train_loader = DataLoader(train_dataset, batch_size=min(batch_size, len(train_dataset)), shuffle=True)
    
    model = GRUAttention().to(device)
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Training
    for epoch in range(epochs):
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb).squeeze(-1)
            loss = loss_fn(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # Accuracy check on the test set
    model.eval()
    test_inputs = scaler.transform(series[-seq_len - len(test_series):-len(test_series)].reshape(-1, 1)).flatten()
    test_actuals = test_series
    
    test_preds = []
    with torch.no_grad():
        current_seq = torch.tensor(test_inputs, dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(device)
        for _ in range(len(test_actuals)):
            out = model(current_seq).item()
            test_preds.append(out)
            # Update sequence for next prediction
            new_seq_val = torch.tensor([[[out]]], dtype=torch.float32).to(device)
            current_seq = torch.cat([current_seq[:, 1:, :], new_seq_val], dim=1)

    test_preds = scaler.inverse_transform(np.array(test_preds).reshape(-1, 1)).flatten()
    
    # Calculate MAPE
    mape = np.mean(np.abs((test_actuals - test_preds) / np.maximum(test_actuals, 1))) * 100
    accuracy = 100 - mape

    # Forecasting for the future
    model.eval()
    input_seq = torch.tensor(scaler.transform(series[-seq_len:].reshape(-1, 1)).flatten(), dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(device)
    future_preds = []
    with torch.no_grad():
        for _ in range(future):
            out = model(input_seq).item()
            future_preds.append(out)
            new_input = torch.cat([input_seq[:, 1:, :], torch.tensor([[[out]]], dtype=torch.float32).to(device)], dim=1)
            input_seq = new_input
            
    future_preds = scaler.inverse_transform(np.array(future_preds).reshape(-1, 1)).flatten()
    future_preds = np.maximum(future_preds, 0)
    
    return np.round(future_preds).tolist(), accuracy

# ----------------------------
# 9. Generate Forecasts per Product
# ----------------------------
def process_product_all(product_id, df, seq_len=30):
    group = df[df["ProductID"] == product_id].sort_values("Date")
    series = group["Total_Quantity_Sold"].values

    if len(series) < 2:
        return None

    daily_forecast, accuracy = train_and_forecast_with_accuracy(series, future=30, seq_len=seq_len)
    weekly_forecast, _ = train_and_forecast_with_accuracy(series, future=7, seq_len=seq_len)
    monthly_forecast, _ = train_and_forecast_with_accuracy(series, future=30, seq_len=seq_len)
    
    print(f"ProductID: {product_id}, Forecast Accuracy: {accuracy:.2f}%")
    
    return {
        "ProductID": product_id,
        "Brand Name": group["Brand Name"].iloc[0],
        "Daily_Forecast": daily_forecast,
        "Weekly_Forecast": weekly_forecast,
        "Monthly_Forecast": monthly_forecast,
        "Accuracy (%)": accuracy
    }

# Run the forecasting in parallel
product_ids = summary_daily_df["ProductID"].unique()
results = Parallel(n_jobs=-1)(delayed(process_product_all)(pid, summary_daily_df) for pid in product_ids)
results = [r for r in results if r is not None]

# Save the product-level forecasts
forecast_df = pd.DataFrame(results)
forecast_df.to_csv("product_forecasts_with_accuracy.csv", index=False)
print("✅ Product-level forecasts saved to product_forecasts_with_accuracy.csv")

# ----------------------------
# 10. Aggregate Forecasts by Brand
# ----------------------------
def aggregate_brand_forecast(forecast_df, col_name, horizon_name):
    expanded = []
    for _, row in forecast_df.iterrows():
        brand_col = row['Brand Name'] if pd.notnull(row['Brand Name']) else 'Unknown'
        for i, val in enumerate(row[col_name]):
            expanded.append({"Brand": brand_col, horizon_name: i + 1, "Forecast_Qty": val})

    expanded_df = pd.DataFrame(expanded)
    brand_forecast = expanded_df.groupby(["Brand", horizon_name], as_index=False)["Forecast_Qty"].sum()
    brand_forecast["Forecast_Qty"] = brand_forecast["Forecast_Qty"].round()
    return brand_forecast

# Generate and save brand-level forecasts
brand_daily_forecast = aggregate_brand_forecast(forecast_df, "Daily_Forecast", "Day")
brand_weekly_forecast = aggregate_brand_forecast(forecast_df, "Weekly_Forecast", "Week")
brand_monthly_forecast = aggregate_brand_forecast(forecast_df, "Monthly_Forecast", "Month")

brand_daily_forecast.to_csv("brand_daily_forecast.csv", index=False)
brand_weekly_forecast.to_csv("brand_weekly_forecast.csv", index=False)
brand_monthly_forecast.to_csv("brand_monthly_forecast.csv", index=False)
print("✅ Brand-level forecasts saved to brand_daily_forecast.csv, brand_weekly_forecast.csv, and brand_monthly_forecast.csv")

ProductID: PVSL0101031125, Forecast Accuracy: 16.96%
ProductID: PVSL0102020126, Forecast Accuracy: 8.40%
ProductID: PVSL0103011025, Forecast Accuracy: -18.14%
ProductID: PVSL0103031025, Forecast Accuracy: 3.73%
ProductID: PVSL0104050126, Forecast Accuracy: 1.92%
ProductID: PVSL0105051025, Forecast Accuracy: -32.72%
ProductID: PVSL0106050126, Forecast Accuracy: -19.32%
ProductID: PVSL0107051025, Forecast Accuracy: -11.33%
ProductID: PVSL0108051025, Forecast Accuracy: -15.43%
ProductID: PVSL0109050126, Forecast Accuracy: -2.23%
ProductID: PVSL0111031025, Forecast Accuracy: 2.43%
ProductID: PVSL0112020925, Forecast Accuracy: -17.69%
ProductID: PVSL0112060925, Forecast Accuracy: -53.94%
ProductID: PVSL0114021125, Forecast Accuracy: 2.47%
ProductID: PVSL0115011125, Forecast Accuracy: -9.41%
ProductID: PVSL0116021125, Forecast Accuracy: 8.76%
ProductID: PVSL0117050126, Forecast Accuracy: -13.58%
ProductID: PVSL0119010925, Forecast Accuracy: 16.87%
ProductID: PVSL0119050925, Forecast Accuracy

In [45]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from joblib import Parallel, delayed
import ast

# Use a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------------------
# 1. Load Data
# ----------------------------
# Load the bills and products datasets
bills_df = pd.read_csv("/kaggle/input/sales-prediction/bills (1).csv")
products_df = pd.read_csv("/kaggle/input/sales-prediction/products.csv")

# ----------------------------
# 2. Preprocess Bills Data
# ----------------------------
# Clean column names by stripping whitespace
bills_df.columns = bills_df.columns.str.strip()

# Convert Product_IDs and Quantities from string representations to lists
bills_df["Product_IDs"] = bills_df["Product_IDs"].apply(ast.literal_eval)
bills_df["Quantities"] = bills_df["Quantities"].apply(ast.literal_eval)

# Explode the DataFrame to have one row per product-quantity pair
bills_df = bills_df.explode(["Product_IDs", "Quantities"])
bills_df.rename(columns={"Product_IDs": "ProductID", "Quantities": "Quantity_Sold"}, inplace=True)

bills_df["ProductID"] = bills_df["ProductID"].astype(str).str.strip()
bills_df["Quantity_Sold"] = bills_df["Quantity_Sold"].astype(int)
bills_df["Date"] = pd.to_datetime(bills_df["Date_Time"]).dt.date

# ----------------------------
# 3. Preprocess Products Data
# ----------------------------
# Clean up column names and ProductID
products_df.columns = products_df.columns.str.strip()
products_df.rename(columns={'Product Code': 'ProductID'}, inplace=True)
products_df["ProductID"] = products_df["ProductID"].astype(str).str.strip()

# --- Find ProductIDs in bills_df that are not in products_df ---
unmatched_ids = set(bills_df["ProductID"]) - set(products_df["ProductID"])
print(f"Found {len(unmatched_ids)} ProductIDs in the bills data that are not in the product data.")
# ----------------------------------------------------------------

# ----------------------------
# 4. Merge DataFrames
# ----------------------------
# Merge the bills and products dataframes
merged_df = bills_df.merge(products_df[['ProductID', 'Brand Name']], on='ProductID', how='left')
merged_df['Brand Name'] = merged_df['Brand Name'].fillna('Unknown')

# ----------------------------
# 5. Create Daily Sales Summary
# ----------------------------
summary_daily_df = (
    merged_df.groupby(["Date", "ProductID", "Brand Name"])["Quantity_Sold"]
    .sum()
    .reset_index()
    .rename(columns={"Quantity_Sold": "Total_Quantity_Sold"})
)

# ----------------------------
# 6. PyTorch Dataset for Time Series
# ----------------------------
class TimeSeriesDataset(Dataset):
    def __init__(self, series, seq_len):
        self.series = series
        self.seq_len = seq_len

    def __len__(self):
        return len(self.series) - self.seq_len

    def __getitem__(self, idx):
        return (
            torch.tensor(self.series[idx:idx+self.seq_len], dtype=torch.float32).unsqueeze(-1),
            torch.tensor(self.series[idx+self.seq_len], dtype=torch.float32)
        )

# ----------------------------
# 7. GRU with Attention Model
# ----------------------------
class GRUAttention(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=64, num_layers=2):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True, dropout=0.2)
        self.attn_fc = nn.Linear(hidden_dim, 1)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        gru_out, _ = self.gru(x)
        attn_weights = torch.softmax(self.attn_fc(gru_out), dim=1)
        context_vector = torch.sum(attn_weights * gru_out, dim=1)
        output = self.fc(context_vector)
        return output

# ----------------------------
# 8. Training, Forecasting, and Accuracy Check
# ----------------------------
def train_and_forecast_with_accuracy(series, future=30, seq_len=30, epochs=100, batch_size=64):
    if len(series) < seq_len + 10: # Ensure enough data for a meaningful split
        return np.zeros(future).tolist(), 0.0

    # Train-test split
    train_series, test_series = train_test_split(series, test_size=0.2, shuffle=False)

    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_series.reshape(-1, 1)).flatten()

    train_dataset = TimeSeriesDataset(train_scaled, seq_len)
    if len(train_dataset) == 0:
        return np.zeros(future).tolist(), 0.0
    
    train_loader = DataLoader(train_dataset, batch_size=min(batch_size, len(train_dataset)), shuffle=True)
    
    model = GRUAttention().to(device)
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Training
    for epoch in range(epochs):
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb).squeeze(-1)
            loss = loss_fn(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # Accuracy check on the test set
    model.eval()
    test_inputs = scaler.transform(series[-seq_len - len(test_series):-len(test_series)].reshape(-1, 1)).flatten()
    test_actuals = test_series
    
    test_preds = []
    with torch.no_grad():
        current_seq = torch.tensor(test_inputs, dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(device)
        for _ in range(len(test_actuals)):
            out = model(current_seq).item()
            test_preds.append(out)
            # Update sequence for next prediction
            new_seq_val = torch.tensor([[[out]]], dtype=torch.float32).to(device)
            current_seq = torch.cat([current_seq[:, 1:, :], new_seq_val], dim=1)

    test_preds = scaler.inverse_transform(np.array(test_preds).reshape(-1, 1)).flatten()
    
    # Calculate MAPE
    mape = np.mean(np.abs((test_actuals - test_preds) / np.maximum(test_actuals, 1))) * 100
    accuracy = 100 - mape

    # Forecasting for the future
    model.eval()
    input_seq = torch.tensor(scaler.transform(series[-seq_len:].reshape(-1, 1)).flatten(), dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(device)
    future_preds = []
    with torch.no_grad():
        for _ in range(future):
            out = model(input_seq).item()
            future_preds.append(out)
            new_input = torch.cat([input_seq[:, 1:, :], torch.tensor([[[out]]], dtype=torch.float32).to(device)], dim=1)
            input_seq = new_input
            
    future_preds = scaler.inverse_transform(np.array(future_preds).reshape(-1, 1)).flatten()
    future_preds = np.maximum(future_preds, 0)
    
    return np.round(future_preds).tolist(), accuracy

# ----------------------------
# 9. Generate Forecasts per Product
# ----------------------------
def process_product_all(product_id, df, seq_len=30):
    group = df[df["ProductID"] == product_id].sort_values("Date")
    series = group["Total_Quantity_Sold"].values

    if len(series) < 2:
        return None

    daily_forecast, accuracy = train_and_forecast_with_accuracy(series, future=30, seq_len=seq_len)
    weekly_forecast, _ = train_and_forecast_with_accuracy(series, future=7, seq_len=seq_len)
    monthly_forecast, _ = train_and_forecast_with_accuracy(series, future=30, seq_len=seq_len)
    
    print(f"ProductID: {product_id}, Forecast Accuracy: {accuracy:.2f}%")
    
    return {
        "ProductID": product_id,
        "Brand Name": group["Brand Name"].iloc[0],
        "Daily_Forecast": daily_forecast,
        "Weekly_Forecast": weekly_forecast,
        "Monthly_Forecast": monthly_forecast,
        "Accuracy (%)": accuracy
    }

# Run the forecasting in parallel
product_ids = summary_daily_df["ProductID"].unique()
results = Parallel(n_jobs=-1)(delayed(process_product_all)(pid, summary_daily_df) for pid in product_ids)
results = [r for r in results if r is not None]

# Save the product-level forecasts
forecast_df = pd.DataFrame(results)
forecast_df.to_csv("product_forecasts_with_accuracy.csv", index=False)
print("✅ Product-level forecasts saved to product_forecasts_with_accuracy.csv")

# ----------------------------
# 10. Aggregate Forecasts by Brand
# ----------------------------
def aggregate_brand_forecast(forecast_df, col_name, horizon_name):
    expanded = []
    for _, row in forecast_df.iterrows():
        brand_col = row['Brand Name'] if pd.notnull(row['Brand Name']) else 'Unknown'
        for i, val in enumerate(row[col_name]):
            expanded.append({"Brand": brand_col, horizon_name: i + 1, "Forecast_Qty": val})

    expanded_df = pd.DataFrame(expanded)
    brand_forecast = expanded_df.groupby(["Brand", horizon_name], as_index=False)["Forecast_Qty"].sum()
    brand_forecast["Forecast_Qty"] = brand_forecast["Forecast_Qty"].round()
    return brand_forecast

# Generate and save brand-level forecasts
brand_daily_forecast = aggregate_brand_forecast(forecast_df, "Daily_Forecast", "Day")
brand_weekly_forecast = aggregate_brand_forecast(forecast_df, "Weekly_Forecast", "Week")
brand_monthly_forecast = aggregate_brand_forecast(forecast_df, "Month_Forecast", "Month")

brand_daily_forecast.to_csv("brand_daily_forecast.csv", index=False)
brand_weekly_forecast.to_csv("brand_weekly_forecast.csv", index=False)
brand_monthly_forecast.to_csv("brand_monthly_forecast.csv", index=False)
print("✅ Brand-level forecasts saved to brand_daily_forecast.csv, brand_weekly_forecast.csv, and brand_monthly_forecast.csv")

Found 1078 ProductIDs in the bills data that are not in the product data.

ProductID: PVSL0905061225, Forecast Accuracy: 11.33%
ProductID: PVSL0906021125, Forecast Accuracy: 8.45%
ProductID: PVSL0906041025, Forecast Accuracy: 5.13%
ProductID: PVSL0906050925, Forecast Accuracy: -12.80%
ProductID: PVSL0907011025, Forecast Accuracy: 0.22%
ProductID: PVSL0907030126, Forecast Accuracy: -36.14%
ProductID: PVSL0907040126, Forecast Accuracy: -20.47%
ProductID: PVSL0907051025, Forecast Accuracy: -33.17%
ProductID: PVSL0908011125, Forecast Accuracy: -13.39%
ProductID: PVSL0908030126, Forecast Accuracy: 11.33%
ProductID: PVSL0908040925, Forecast Accuracy: -4.72%
ProductID: PVSL0908060925, Forecast Accuracy: -3.77%
ProductID: PVSL0909011125, Forecast Accuracy: -27.22%
ProductID: PVSL0909031125, Forecast Accuracy: 8.21%
ProductID: PVSL0909050925, Forecast Accuracy: 2.35%
ProductID: PVSL0909061125, Forecast Accuracy: -28.53%
ProductID: PVSL0910030126, Forecast Accuracy: -8.35%
ProductID: PVSL0910051

KeyboardInterrupt: 

In [46]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from joblib import Parallel, delayed
import ast

# Use a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------------------
# 1. Load Data
# ----------------------------
# Load the bills and products datasets
bills_df = pd.read_csv("/kaggle/input/sales-prediction/bills (1).csv")
products_df = pd.read_csv("/kaggle/input/sales-prediction/products.csv")

# ----------------------------
# 2. Preprocess Bills Data
# ----------------------------
# Clean column names by stripping whitespace
bills_df.columns = bills_df.columns.str.strip()

# Convert Product_IDs and Quantities from string representations to lists
bills_df["Product_IDs"] = bills_df["Product_IDs"].apply(ast.literal_eval)
bills_df["Quantities"] = bills_df["Quantities"].apply(ast.literal_eval)


# Explode the DataFrame to have one row per product-quantity pair
bills_df = bills_df.explode(["Product_IDs", "Quantities"])
bills_df.rename(columns={"Product_IDs": "ProductID", "Quantities": "Quantity_Sold"}, inplace=True)


bills_df["ProductID"] = bills_df["ProductID"].astype(str).str.strip()
bills_df["Quantity_Sold"] = bills_df["Quantity_Sold"].astype(int)

bills_df["Date"] = pd.to_datetime(bills_df["Date_Time"]).dt.date

# ----------------------------
# 3. Preprocess Products Data
# ----------------------------
# Clean up column names and ProductID
products_df.columns = products_df.columns.str.strip()
products_df.rename(columns={'Product Code': 'ProductID'}, inplace=True)
products_df["ProductID"] = products_df["ProductID"].astype(str).str.strip()

# --- Find ProductIDs in bills_df that are not in products_df ---
unmatched_ids = set(bills_df["ProductID"]) - set(products_df["ProductID"])
if unmatched_ids:
    print(f"Found {len(unmatched_ids)} ProductIDs in the bills data that are not in the product data.")
# ----------------------------------------------------------------

# ----------------------------
# 4. Merge DataFrames
# ----------------------------
# Merge the bills and products dataframes
merged_df = bills_df.merge(products_df[['ProductID', 'Brand Name']], on='ProductID', how='left')
merged_df['Brand Name'] = merged_df['Brand Name'].fillna('Unknown')

# ----------------------------
# 5. Create Daily Sales Summary and Feature Engineering
# ----------------------------
summary_daily_df = (
    merged_df.groupby(["Date", "ProductID", "Brand Name"])["Quantity_Sold"]
    .sum()
    .reset_index()
    .rename(columns={"Quantity_Sold": "Total_Quantity_Sold"})
)
# --- Feature Engineering ---
summary_daily_df['Date'] = pd.to_datetime(summary_daily_df['Date'])
summary_daily_df['dayofweek'] = summary_daily_df['Date'].dt.dayofweek
summary_daily_df['dayofyear'] = summary_daily_df['Date'].dt.dayofyear
summary_daily_df['month'] = summary_daily_df['Date'].dt.month
summary_daily_df['weekofyear'] = summary_daily_df['Date'].dt.isocalendar().week.astype(int)


# ----------------------------
# 6. PyTorch Dataset for Time Series
# ----------------------------
class TimeSeriesDataset(Dataset):
    def __init__(self, features, targets, seq_len):
        self.features = features
        self.targets = targets
        self.seq_len = seq_len

    def __len__(self):
        return len(self.features) - self.seq_len

    def __getitem__(self, idx):
        return (
            torch.tensor(self.features[idx:idx+self.seq_len], dtype=torch.float32),
            torch.tensor(self.targets[idx+self.seq_len], dtype=torch.float32)
        )

# ----------------------------
# 7. LSTM Model
# ----------------------------
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=3):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=0.3)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        output = self.fc(lstm_out[:, -1, :])
        return output

# ----------------------------
# 8. Training, Forecasting, and Accuracy Check
# ----------------------------
def train_and_forecast_with_accuracy(series_df, future=30, seq_len=30, epochs=150, batch_size=64):
    if len(series_df) < seq_len + 15: # Ensure enough data
        return np.zeros(future).tolist(), 0.0

    features = series_df[['Total_Quantity_Sold', 'dayofweek', 'dayofyear', 'month', 'weekofyear']].values
    targets = series_df['Total_Quantity_Sold'].values

    # Train-test split
    train_features, test_features, train_targets, test_targets = train_test_split(
        features, targets, test_size=0.2, shuffle=False
    )

    scaler = MinMaxScaler()
    train_features_scaled = scaler.fit_transform(train_features)
    
    train_dataset = TimeSeriesDataset(train_features_scaled, train_targets, seq_len)
    if len(train_dataset) == 0:
        return np.zeros(future).tolist(), 0.0

    train_loader = DataLoader(train_dataset, batch_size=min(batch_size, len(train_dataset)), shuffle=True)
    
    model = LSTMModel(input_dim=train_features.shape[1]).to(device)
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Training
    for epoch in range(epochs):
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb).squeeze(-1)
            loss = loss_fn(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # Accuracy check
    model.eval()
    test_inputs = scaler.transform(features[-seq_len - len(test_features):-len(test_features)])
    test_actuals = test_targets
    
    test_preds = []
    with torch.no_grad():
        current_seq = torch.tensor(test_inputs, dtype=torch.float32).unsqueeze(0).to(device)
        for _ in range(len(test_actuals)):
            out = model(current_seq).item()
            test_preds.append(out)
            # Create the next input sequence
            last_features = current_seq[:, -1, 1:].cpu().numpy() # Get the time features
            new_row = np.hstack(([out], last_features[0])).reshape(1, -1)
            new_row_scaled = scaler.transform(new_row)
            new_seq_val = torch.tensor(new_row_scaled, dtype=torch.float32).unsqueeze(0).to(device)
            current_seq = torch.cat([current_seq[:, 1:, :], new_seq_val], dim=1)

    # Inverse transform only the sales predictions
    sales_scaler = MinMaxScaler()
    sales_scaler.min_, sales_scaler.scale_ = scaler.min_[0], scaler.scale_[0]
    test_preds = sales_scaler.inverse_transform(np.array(test_preds).reshape(-1, 1)).flatten()
    
    mape = np.mean(np.abs((test_actuals - test_preds) / np.maximum(test_actuals, 1))) * 100
    accuracy = 100 - mape

    # Forecasting
    model.eval()
    input_seq_features = scaler.transform(features[-seq_len:])
    future_preds = []
    with torch.no_grad():
        current_seq = torch.tensor(input_seq_features, dtype=torch.float32).unsqueeze(0).to(device)
        for _ in range(future):
            out = model(current_seq).item()
            future_preds.append(out)
            last_features = current_seq[:, -1, 1:].cpu().numpy() # Get the time features
            new_row = np.hstack(([out], last_features[0])).reshape(1, -1)
            new_row_scaled = scaler.transform(new_row)
            new_seq_val = torch.tensor(new_row_scaled, dtype=torch.float32).unsqueeze(0).to(device)
            current_seq = torch.cat([current_seq[:, 1:, :], new_seq_val], dim=1)
            
    future_preds = sales_scaler.inverse_transform(np.array(future_preds).reshape(-1, 1)).flatten()
    future_preds = np.maximum(future_preds, 0)
    
    return np.round(future_preds).tolist(), accuracy

# ----------------------------
# 9. Generate Forecasts per Product
# ----------------------------
def process_product_all(product_id, df, seq_len=30):
    group = df[df["ProductID"] == product_id].sort_values("Date")
    
    if len(group) < 2:
        return None

    daily_forecast, accuracy = train_and_forecast_with_accuracy(group, future=30, seq_len=seq_len)
    weekly_forecast, _ = train_and_forecast_with_accuracy(group, future=7, seq_len=seq_len)
    monthly_forecast, _ = train_and_forecast_with_accuracy(group, future=30, seq_len=seq_len)
    
    print(f"ProductID: {product_id}, Forecast Accuracy: {accuracy:.2f}%")
    
    return {
        "ProductID": product_id,
        "Brand Name": group["Brand Name"].iloc[0],
        "Daily_Forecast": daily_forecast,
        "Weekly_Forecast": weekly_forecast,
        "Monthly_Forecast": monthly_forecast,
        "Accuracy (%)": accuracy
    }

# Run the forecasting in parallel
product_ids = summary_daily_df["ProductID"].unique()
results = Parallel(n_jobs=-1)(delayed(process_product_all)(pid, summary_daily_df) for pid in product_ids)
results = [r for r in results if r is not None]

# Save the product-level forecasts
forecast_df = pd.DataFrame(results)
forecast_df.to_csv("product_forecasts_with_accuracy.csv", index=False)
print("✅ Product-level forecasts saved to product_forecasts_with_accuracy.csv")

# ----------------------------
# 10. Aggregate Forecasts by Brand
# ----------------------------
def aggregate_brand_forecast(forecast_df, col_name, horizon_name):
    expanded = []
    for _, row in forecast_df.iterrows():
        brand_col = row['Brand Name'] if pd.notnull(row['Brand Name']) else 'Unknown'
        for i, val in enumerate(row[col_name]):
            expanded.append({"Brand": brand_col, horizon_name: i + 1, "Forecast_Qty": val})

    expanded_df = pd.DataFrame(expanded)
    brand_forecast = expanded_df.groupby(["Brand", horizon_name], as_index=False)["Forecast_Qty"].sum()
    brand_forecast["Forecast_Qty"] = brand_forecast["Forecast_Qty"].round()
    return brand_forecast

# Generate and save brand-level forecasts
brand_daily_forecast = aggregate_brand_forecast(forecast_df, "Daily_Forecast", "Day")
brand_weekly_forecast = aggregate_brand_forecast(forecast_df, "Weekly_Forecast", "Week")
brand_monthly_forecast = aggregate_brand_forecast(forecast_df, "Month_Forecast", "Month")

brand_daily_forecast.to_csv("brand_daily_forecast.csv", index=False)
brand_weekly_forecast.to_csv("brand_weekly_forecast.csv", index=False)
brand_monthly_forecast.to_csv("brand_monthly_forecast.csv", index=False)
print("✅ Brand-level forecasts saved to brand_daily_forecast.csv, brand_weekly_forecast.csv, and brand_monthly_forecast.csv")

Found 1078 ProductIDs in the bills data that are not in the product data.


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from joblib import Parallel, delayed
import ast

# ----------------------------
# 1. Load Data
# ----------------------------
# Load the bills and products datasets
bills_df = pd.read_csv("/kaggle/input/sales-prediction/bills (1).csv")
products_df = pd.read_csv("/kaggle/input/sales-prediction/products.csv")

# ----------------------------
# 2. Preprocess Data
# ----------------------------
# Clean column names by stripping whitespace
bills_df.columns = bills_df.columns.str.strip()
products_df.columns = products_df.columns.str.strip()

# Clean and process bills data
bills_df["Product_IDs"] = bills_df["Product_IDs"].apply(ast.literal_eval)
bills_df["Quantities"] = bills_df["Quantities"].apply(ast.literal_eval)
bills_df = bills_df.explode(["Product_IDs", "Quantities"])
bills_df.rename(columns={"Product_IDs": "ProductID", "Quantities": "Quantity_Sold"}, inplace=True)
bills_df["ProductID"] = bills_df["ProductID"].astype(str).str.strip()
bills_df["Quantity_Sold"] = bills_df["Quantity_Sold"].astype(int)
bills_df["Date"] = pd.to_datetime(bills_df["Date_Time"]).dt.date

# Clean and process products data
products_df.rename(columns={'Product Code': 'ProductID'}, inplace=True)
products_df["ProductID"] = products_df["ProductID"].astype(str).str.strip()

# ----------------------------
# 3. Merge DataFrames
# ----------------------------
# Merge and handle products that are in bills but not in the product list
merged_df = bills_df.merge(products_df[['ProductID', 'Brand Name']], on='ProductID', how='left')
merged_df['Brand Name'] = merged_df['Brand Name'].fillna('Unknown')

# ----------------------------
# 4. Create Daily Sales Summary
# ----------------------------
daily_sales = (
    merged_df.groupby(["Date", "ProductID", "Brand Name"])["Quantity_Sold"]
    .sum()
    .reset_index()
    .rename(columns={"Quantity_Sold": "Total_Quantity_Sold"})
)
daily_sales['Date'] = pd.to_datetime(daily_sales['Date'])

# ----------------------------
# 5. Feature Engineering Function
# ----------------------------
def create_features(df):
    """Create time series features based on the date index."""
    df = df.copy()
    df['dayofweek'] = df['Date'].dt.dayofweek
    df['dayofyear'] = df['Date'].dt.dayofyear
    df['month'] = df['Date'].dt.month
    df['weekofyear'] = df['Date'].dt.isocalendar().week.astype(int)
    
    # Lag and Rolling Features
    for lag in [1, 2, 3, 7]:
        df[f'lag_{lag}'] = df['Total_Quantity_Sold'].shift(lag)
    for window in [7, 14]:
        df[f'rolling_mean_{window}'] = df['Total_Quantity_Sold'].shift(1).rolling(window).mean()
        
    return df.fillna(-1) # Fill NaNs created by lags/rolling windows

# ----------------------------
# 6. Training and Forecasting Function
# ----------------------------
def train_and_forecast_lgbm(product_df, future_days=30):
    if len(product_df) < 30: # Need enough data to create features and train
        return np.zeros(future_days).tolist(), 0.0

    # Create a full date range to handle days with no sales
    full_date_range = pd.date_range(start=product_df['Date'].min(), end=product_df['Date'].max())
    product_df = product_df.set_index('Date').reindex(full_date_range).fillna(0).reset_index()
    product_df.rename(columns={'index': 'Date'}, inplace=True)

    # Create features
    data = create_features(product_df)
    
    # Split data for training and accuracy check
    train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)
    
    features = ['dayofweek', 'dayofyear', 'month', 'weekofyear', 
                'lag_1', 'lag_2', 'lag_3', 'lag_7', 'rolling_mean_7', 'rolling_mean_14']
    target = 'Total_Quantity_Sold'

    X_train, y_train = train_data[features], train_data[target]
    X_test, y_test = test_data[features], test_data[target]

    # Train the LightGBM model
    model = lgb.LGBMRegressor(
        objective='regression_l1',
        n_estimators=1000,
        learning_rate=0.05,
        num_leaves=31,
        max_depth=-1,
        min_child_samples=20,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], 
              callbacks=[lgb.early_stopping(100, verbose=False)])

    # Accuracy check
    preds = model.predict(X_test)
    mape = np.mean(np.abs((y_test - preds) / np.maximum(y_test, 1))) * 100
    accuracy = 100 - mape

    # Forecasting
    future_preds = []
    last_known_data = data.tail(14).copy() # Use last 14 days to generate features

    for _ in range(future_days):
        last_date = last_known_data['Date'].iloc[-1]
        next_date = last_date + pd.Timedelta(days=1)
        
        # Create a new row for the next day's prediction
        next_day_features_df = pd.DataFrame([{'Date': next_date, 'Total_Quantity_Sold': 0}])
        
        # Create features for the next day
        combined_data = pd.concat([last_known_data, next_day_features_df], ignore_index=True)
        combined_features = create_features(combined_data)
        
        next_day_features = combined_features[features].tail(1)
        
        # Predict
        prediction = model.predict(next_day_features)[0]
        prediction = max(0, prediction) # Ensure non-negative sales
        
        # Add the prediction back for the next loop
        new_row = pd.DataFrame([{'Date': next_date, 'Total_Quantity_Sold': prediction}])
        last_known_data = pd.concat([last_known_data, new_row], ignore_index=True)
        future_preds.append(prediction)

    return np.round(future_preds).tolist(), accuracy

# ----------------------------
# 7. Generate Forecasts per Product
# ----------------------------
def process_product(product_id, df):
    group = df[df["ProductID"] == product_id].sort_values("Date")
    
    if len(group) < 2:
        return None

    daily_forecast, accuracy = train_and_forecast_lgbm(group.copy(), future_days=30)
    weekly_forecast = daily_forecast[:7]
    monthly_forecast = daily_forecast
    
    print(f"ProductID: {product_id}, Brand: {group['Brand Name'].iloc[0]}, Forecast Accuracy: {accuracy:.2f}%")
    
    return {
        "ProductID": product_id,
        "Brand Name": group["Brand Name"].iloc[0],
        "Daily_Forecast": daily_forecast,
        "Weekly_Forecast": weekly_forecast,
        "Monthly_Forecast": monthly_forecast,
        "Accuracy (%)": accuracy
    }

# Run forecasting in parallel
product_ids = daily_sales["ProductID"].unique()
results = Parallel(n_jobs=-1)(delayed(process_product)(pid, daily_sales) for pid in product_ids)
results = [r for r in results if r is not None]

# Save forecasts
forecast_df = pd.DataFrame(results)
forecast_df.to_csv("product_forecasts_with_accuracy.csv", index=False)
print("\n✅ Product-level forecasts saved to product_forecasts_with_accuracy.csv")

# ----------------------------
# 8. Aggregate Forecasts by Brand
# ----------------------------
def aggregate_brand_forecast(forecast_df, col_name, horizon_name):
    expanded = []
    for _, row in forecast_df.iterrows():
        brand_col = row['Brand Name'] if pd.notnull(row['Brand Name']) else 'Unknown'
        for i, val in enumerate(row[col_name]):
            expanded.append({"Brand": brand_col, horizon_name: i + 1, "Forecast_Qty": val})

    expanded_df = pd.DataFrame(expanded)
    brand_forecast = expanded_df.groupby(["Brand", horizon_name], as_index=False)["Forecast_Qty"].sum()
    brand_forecast["Forecast_Qty"] = brand_forecast["Forecast_Qty"].round()
    return brand_forecast

# Generate and save brand-level forecasts
brand_daily_forecast = aggregate_brand_forecast(forecast_df, "Daily_Forecast", "Day")
brand_weekly_forecast = aggregate_brand_forecast(forecast_df, "Weekly_Forecast", "Week")
brand_monthly_forecast = aggregate_brand_forecast(forecast_df, "Monthly_Forecast", "Month")

brand_daily_forecast.to_csv("brand_daily_forecast.csv", index=False)
brand_weekly_forecast.to_csv("brand_weekly_forecast.csv", index=False)
brand_monthly_forecast.to_csv("brand_monthly_forecast.csv", index=False)
print("✅ Brand-level forecasts saved.")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.056199 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 853
[LightGBM] [Info] Number of data points in the train set: 900, number of used features: 10
[LightGBM] [Info] Start training from score 23.000000
ProductID: PVSL0101021125, Brand: Unknown, Forecast Accuracy: -10.46%
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.070185 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 857
[LightGBM] [Info] Number of data points in the train set: 900, number of used features: 10
[LightGBM] [Info] Start training from score 23.000000
ProductID: PVSL0102020126, Brand: Unknown, Forecast Accuracy: -33.63%
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.059183 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] 